# Leukemia Classifier

In [ ]:
import pandas

In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files

In [ ]:
files.upload() #upload your Kaggle API

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"gargmanas","key":"6b301da73a40c24b04b765960a3a534b"}'}

In [ ]:
!mkdir ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d andrewmvd/leukemia-classification

 98% 850M/867M [00:11<00:00, 73.4MB/s]
100% 867M/867M [00:12<00:00, 75.7MB/s]


In [ ]:
!unzip leukemia-classification.zip

Archive:  leukemia-classification.zip
  inflating: C-NMC_Leukemia/testing_data/C-NMC_test_final_phase_data/1.bmp  
  inflating: C-NMC_Leukemia/testing_data/C-NMC_test_final_phase_data/10.bmp  
  inflating: C-NMC_Leukemia/testing_data/C-NMC_test_final_phase_data/100.bmp  
  inflating: C-NMC_Leukemia/testing_data/C-NMC_test_final_phase_data/1000.bmp  
  inflating: C-NMC_Leukemia/testing_data/C-NMC_test_final_phase_data/1001.bmp  
  inflating: C-NMC_Leukemia/testing_data/C-NMC_test_final_phase_data/1002.bmp  
  inflating: C-NMC_Leukemia/testing_data/C-NMC_test_final_phase_data/1003.bmp  
  inflating: C-NMC_Leukemia/testing_data/C-NMC_test_final_phase_data/1004.bmp  
  inflating: C-NMC_Leukemia/testing_data/C-NMC_test_final_phase_data/1005.bmp  
  inflating: C-NMC_Leukemia/testing_data/C-NMC_test_final_phase_data/1006.bmp  
  inflating: C-NMC_Leukemia/testing_data/C-NMC_test_final_phase_data/1007.bmp  
  inflating: C-NMC_Leukemia/testing_data/C-NMC_test_final_phase_data/1008.bmp  
  inflat

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.applications import VGG19, imagenet_utils
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import img_to_array, load_img, ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import numpy as np
import os

In [ ]:
imageData = []
labels = []

In [ ]:
import progressbar

In [ ]:
ls

C-NMC_Leukemia/  kaggle.json  leukemia-classification.zip  sample_data/


In [ ]:
### progressbar
widgets = ["Image Processing ", progressbar.Percentage(), " ", progressbar.Bar(), " ", progressbar.ETA()]
Pbar = progressbar.ProgressBar(maxval=15115, widgets=widgets).start()

import os
n = 0
for dirname, _, filenames in os.walk('./C-NMC_Leukemia'):
    for filename in filenames:
        path = os.path.join(dirname, filename)
        Splited_Path = path.split("/")
        if Splited_Path[-4] == "training_data":
            if Splited_Path[-2] == "all":
                labels.append(1)
            if Splited_Path[-2] == "hem":
                labels.append(0)
            image = load_img(path, target_size=(224, 224))
            image = img_to_array(image)
            image = imagenet_utils.preprocess_input(image)
            imageData.append(image)
        n = n + 1
        Pbar.update(n)
Pbar.finish()
        

Image Processing 100% |########################################| Time:  0:00:16


In [ ]:
imageData = np.array(imageData)
labels = np.array(labels)

In [ ]:
from sklearn.preprocessing import LabelBinarizer
Labels = LabelBinarizer()

In [ ]:
Labels = Labels.fit_transform(labels)
Labels = to_categorical(Labels)

In [ ]:
trainX, testX, trainY, testY = train_test_split(imageData, Labels, test_size = 0.2)

In [ ]:
### start your tf

In [ ]:
baseModel = VGG19(include_top=False, input_tensor=Input(shape=(224,224,3)))

80142336/80134624 [==============================] - 1s 0us/step


In [ ]:
headModel = baseModel.output
headModel = Flatten(name = "flatten")(headModel)
headModel = Dense(256,activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)

#output layer
headModel = Dense(2, activation="softmax")(headModel)
model = Model(inputs = baseModel.input, outputs = headModel)

In [ ]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

sample_data/


In [ ]:
for layers in baseModel.layers:
  layers.trainable = False

In [ ]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

In [ ]:
imageAug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1, fill_mode="nearest", height_shift_range=0.1, zoom_range=0.2, horizontal_flip=True)

In [ ]:
model.compile(loss = "categorical_crossentropy", optimizer="rmsprop", metrics=["accuracy"])
print("Model is going to  train...")
H = model.fit(imageAug.flow(trainX, trainY, batch_size=32),validation_data=(testX, testY),epochs = 10, verbose =1)

Model is going to  train...
Epoch 1/10
267/267 [==============================] - 97s 364ms/step - loss: 1.9981 - accuracy: 0.7404 - val_loss: 0.5372 - val_accuracy: 0.7867
Epoch 2/10
267/267 [==============================] - 98s 367ms/step - loss: 0.5895 - accuracy: 0.7685 - val_loss: 0.5557 - val_accuracy: 0.7932
Epoch 3/10
267/267 [==============================] - 98s 369ms/step - loss: 0.5500 - accuracy: 0.7790 - val_loss: 0.4742 - val_accuracy: 0.7923
Epoch 4/10
267/267 [==============================] - 99s 369ms/step - loss: 0.5419 - accuracy: 0.7742 - val_loss: 0.5814 - val_accuracy: 0.7736
Epoch 5/10
267/267 [==============================] - 99s 369ms/step - loss: 0.5471 - accuracy: 0.7824 - val_loss: 0.5254 - val_accuracy: 0.7731
Epoch 6/10
267/267 [==============================] - 99s 370ms/step - loss: 0.5419 - accuracy: 0.7819 - val_loss: 0.4793 - val_accuracy: 0.8012
Epoch 7/10
267/267 [==============================] - 99s 371ms/step - loss: 0.5368 - accuracy: 0.7737

In [ ]:
model.save("Leukemia_trained_model.h5")

In [ ]:
ls

C-NMC_Leukemia/  leukemia-classification.zip  sample_data/
kaggle.json      Leukemia_trained_model.h5


# Importing Pre-trained models

In [ ]:
#from google.colab import files

In [ ]:
#files.upload()

In [ ]:
#from tensorflow.keras.models import load_model

In [ ]:
#model = load_model("Leukemia_trained_model.h5")

In [ ]:
#model.summary()

# Making prediction

In [ ]:
from PIL import Image 

In [ ]:
image = load_img("/content/C-NMC_Leukemia/testing_data/C-NMC_test_final_phase_data/100.bmp", target_size=(224,224))
image = img_to_array(image)
image = np.expand_dims(image, axis=0)
image = imagenet_utils.preprocess_input(image)
pred = model.predict(image)[0]
if pred[0] > pred[1]:
  print("This image is 'Hem' image")
else:
  print("This image is 'All' image")

This image is 'All' image
